## Gaussian Fit






## Skeleton of the Interpolation Module


## The Model Class: 
This class handles basic IO of the Model Data extracting grid quintities and variables as selected by
the user to be used in the interpolation.


In [1]:
from netCDF4 import Dataset
import matplotlib.pyplot as plt
%matplotlib inline  
import pandas as pd
import numpy as np
import pandas as pd
from pandas import DataFrame
from datetime import datetime
import time
import os
import sys
import multiprocessing
from functools import partial
from scipy.signal import savgol_filter
import warnings
warnings.filterwarnings('ignore')

startTime = 0

In [2]:

class Model:
    def __init__(self,name, maxAltitude, minAltitude):
        self.name=name
        self.maxAltitude=maxAltitude
        self.minAltitude=minAltitude
        self.dt=0.0



    def readGrid(self,name):
        TIEGCM=Dataset(name+".nc")
        glat=TIEGCM.variables['lat'][:]
        glon=TIEGCM.variables['lon'][:]
        glev=TIEGCM.variables['lev'][:]
        gtime=TIEGCM.variables['time'][:] 
        zg=TIEGCM.variables['ZGMID'][:]
        TIEGCM.close()
        self.dt= gtime[2]- gtime[1]
       
        return np.asarray(gtime),np.asarray(glat),np.asarray(glon),np.asarray(glev),np.asarray(zg)

    def readVar(self,model,name):
        TIEGCM=Dataset(model+".nc")
        var=TIEGCM.variables[name][:]
        TIEGCM.close()
        return var


## The Orbit Class: 
This class  This class handles basic IO of Daedalus' orbit allocating arrays for the spatial components. The longitudinal component is matched to TIEGCM's  and points in a specific altitudinal range are extracted to be passed to the interpolation routine. There is also an option for creating a Rocket orbit for getting 
vertical profiles of the said model.

In [3]:

class Orbit:
    def __init__(self,name):
        self.name=name
        self.dt=1/16



    def createorbit(self,filename,minAlt,maxAlt,outfile,Save=False):
        orbit=Dataset(filename+".nc","r")
        daed_lat_temp = orbit.variables['lat'][:]
        daed_lon_temp = orbit.variables['lon'][:]
        daed_alt_temp = orbit.variables['altitude'][:]
        daed_time_temp = orbit.variables['time'][:] #unix time
        
        global startTime
        startTime = daed_time_temp[0]
        

#         self.dt=daed_time_temp[2]-daed_time_temp[1]
        self.Save=Save
        
        if self.Save==True:
            self.OpenFile(outfile,daed_time_temp,daed_lat_temp,daed_lon_temp,daed_alt_temp)
        
        stop=False
        counter=0
        for i in range(0,len(daed_alt_temp)):
            if (daed_alt_temp[i] < maxAlt and daed_alt_temp[i] > minAlt):
                counter=counter+1

        
        
        daed_lat=np.zeros((counter))
        daed_lon=np.zeros((counter))
        daed_alt=np.zeros((counter))
        daed_time=np.zeros((counter),dtype=datetime)
        index=[None]*counter
        int_final=[None]*len(daed_alt_temp)


   
        counter2=0
        
        for i in range(0,len(daed_alt_temp)):
    
            if (daed_alt_temp[i] < maxAlt and daed_alt_temp[i] > minAlt):
         
                daed_time[counter2]=daed_time_temp[i]
                daed_lat[counter2]=daed_lat_temp[i]
                if max(daed_lon_temp)>180:
                    daed_lon[counter2]=daed_lon_temp[i]-180 #match TIEGCM grid
                else:
                    daed_lon[counter2]=daed_lon_temp[i]
                daed_alt[counter2]=daed_alt_temp[i]
                
                index[counter2]=i          #keep indices for merging data
                counter2=counter2+1
        

        return(daed_time,daed_lat,daed_lon,daed_alt,index,int_final)

    def RockeOrbit(self,lat,lon,alt,nop,dz):
        Rlat=np.zeros(nop)
        Rlon=np.zeros(nop)
        Ralt=np.zeros(nop)

        Rlat[:]=lat
        Rlon[:]=lon
        Ralt[0]=alt
        for i in range(1,nop):
            Ralt[i]=Ralt[i-1]+dz

        return Rlat,Rlon,Ralt

    def mergeData(self,index,int_final,m):
        for i in range(0,len(m)):
            int_final[index[i]]=m[i]

        return int_final    



    def OpenFile(self,outfile,time,lat,lon,alt):
        print("Output File Path:",outfile)
        ncout = Dataset(outfile, "w", format="NETCDF4")    
        ncout.createDimension("time", len(time))
        xaxis=np.arange(0,len(time)) #time axis based on Daedalus Sampling rate
        data = ncout.createVariable("time","f4","time")
        data[:]=xaxis
        ncout.createDimension("lat",len(lat))
        data = ncout.createVariable("lat","f4","lat")
        data[:]=lat
        ncout.createDimension("lon",len(lon))
        data = ncout.createVariable("lon","f4","lon")
        data[:]=lon
        ncout.createDimension("altitude",len(alt))
        data = ncout.createVariable("altitude","f4","time")
        data[:]=alt
        ncout.close()   
        return True



Functions used by the Interpolation Module
The geod2geo_lat converts geodetic latitude to geographic latitude

In [4]:
def geod_lat2geo_lat(phi):

    # calculate geocentric latitude from geodetic latitude
    # according to WGS 84
    a = 6378137  # meter semi major axis of earth
    f = 1 / 298.257  # flattening
    b = a - f * a  # semi minor axis
    e = ((a ** 2 - b ** 2) ** (1 / 2)) / a
    phi_rad = np.deg2rad(phi)
    geo_lat = np.arctan((1 - e ** 2) * np.tan(phi_rad))
    geo_lat = np.rad2deg(geo_lat)
    return geo_lat  # in degrees

The function local is used to find the local neighbors of a specific component in a non eqwually spaced array 
such as the pressure levels in the TIEGCM

In [5]:

def local(dim1,y,x):
    local_pos=0
    for i in range(0,len(x)-1):
        if y >= x[i] and y < x[i+1]:
            local_pos=i
            return (local_pos)
    return (local_pos)

The single threaded inteprolation core routine which iterates over Daedalus orbit and performs the interpolation.
At every spatial position weights are assigned to the  8 local grid neighbours and then the interpolated value is calculated using trilinear interpolation.

In [6]:

from tqdm import tqdm
def Interpolate_Serial(glat,glon,glev,dtime,dlat,dlon,dalt,zg,ne,Plot=False,thisLabel="TN"):
    counter=22
    deltaphi= np.abs(glon[2]-glon[1])
    deltatheta=np.abs(glat[2]-glat[1])
    Re=6378137.0/1e3
    # arc_theta=2*np.pi*Re*(deltatheta/360)
    # arc_phi=2*np.pi*Re*(deltaphi/360)

    m=np.zeros((len(dalt)))
    tn=np.zeros((len(dalt)))
    pn=np.zeros((len(dalt)))

    for i in range(0,len(dalt)):


        
        phi_local=local(len(glon),dlon[i],glon)
        theta_local=local(len(glat),dlat[i],glat)


        if dlon[i] >= 177.5 :
            phi_local=len(glon)-2
            lonbnd=1

        if dlon[i] <= -177.5:
            phi_local=1

        if dlat[i] >= 87.5:
            theta_local=len(glat)-2
            latbnd=1

        if dlat[i] <= -87.5:
            theta_local=0


        alts=zg[counter,:,theta_local,phi_local]/1e5
        r_local=local(len(alts),dalt[i],alts)
        deltarho=alts[r_local+1]-alts[r_local]
        dx=(((dalt[i]-alts[r_local])/deltarho))
        dy=(((dlat[i]-glat[theta_local])/deltatheta))
        dz=np.abs(((dlon[i]-glon[phi_local])/deltaphi))

        # Calculate Neighbors
        pn[i]=ne[counter,r_local,theta_local,phi_local]

        
        w1=np.abs((1-dx)*(1-dy)*(1-dz))
        w2=np.abs((dx)*(1-dy)*(1-dz))
        w3=np.abs((1-dx)*(dy)*(1-dz))
        w4=np.abs((dx)*(dy)*(1-dz))
        w5=np.abs((1-dx)*(1-dy)*(dz))
        w6=np.abs((dx)*(1-dy)*(dz))
        w7=np.abs((1-dx)*(dy)*(dz))
        w8=np.abs((dx)*(dy)*(dz))

        
        m[i]=0.0
        m[i]=       ne[counter,r_local,theta_local,phi_local]*w1
        m[i]=m[i]+  ne[counter,r_local+1,theta_local,phi_local]*w2
        m[i]=m[i]+  ne[counter,r_local,theta_local+1,phi_local]*w3
        m[i]=m[i]+  ne[counter,r_local+1,theta_local+1,phi_local]*w4

        m[i]=m[i]+  ne[counter,r_local,theta_local,phi_local+1]*w5
        m[i]=m[i]+  ne[counter,r_local+1,theta_local,phi_local+1]*w6
        m[i]=m[i]+  ne[counter,r_local,theta_local+1,phi_local+1]*w7
        m[i]=m[i]+  ne[counter,r_local+1,theta_local+1,phi_local+1]*w8


    min=np.zeros((len(dtime)))
    for i in range(0,len(dtime)):
        min[i] = (dtime[i] - startTime)/1000/60
        
    if Plot== True:
        fig = plt.figure()
        fig.set_figheight(10)
        fig.set_figwidth(7)
        
        ax1 = fig.add_subplot(311)
        ax2 = fig.add_subplot(312)
        ax3 = fig.add_subplot(313)


        ax1.plot(min,m)
        ax2.plot(min,dlat)
        ax3.plot(min,dalt)
        
        ax1.set_ylabel(thisLabel, fontsize=14)
        ax2.set_ylabel("Lat [deg]", fontsize=14)
        ax3.set_ylabel("Altitude [km]", fontsize=14)
        
        plt.suptitle('Interpolated Data (from ' + str(datetime.utcfromtimestamp(startTime/1000.0).strftime('%Y-%m-%d %H:%M:%S')) + ')', fontsize=24)
        plt.xlabel("Time (min)", fontsize=14)
        plt.show() 

    return m



def Write(outfile,m,varname):

    ncout = Dataset(outfile, "a", format="NETCDF4")
    data = ncout.createVariable(varname,"f4",('time'))
    data[:]=m
    ncout.close()
    
    
    
from ipywidgets import*
import ipywidgets as widgets
import warnings
import sys
sys.path.insert(1, "../SourceCode")
import DaedalusGlobals as DaedalusGlobals
import glob
import os


def LoadGUI():
   
    layout1 = {'width':'850px'}
    style1 = {'description_width':'150px'}
    
    tiegcm_file=widgets.Dropdown(options=sorted(glob.glob(DaedalusGlobals.TIEGCM_Files_Path +"TIEGCM_EVT1_2015_StPatricksDay_Rerun/*.nc")), description='TIE-GCM files: ', style=style1, layout=layout1)
    orbit_file=widgets.Dropdown(options=sorted(glob.glob(DaedalusGlobals.Orbit_Files_Path +"DAED_ORB_Evt0_LLA*.nc")), description='Orbit files: ', style=style1, layout=layout1)
    saveOption= widgets.Checkbox(value=False,  description="Save Output",style=style1, layout=layout1)
    TGvar=widgets.Dropdown(
            options=['All','XNMBAR','TI','TN','OP','DEN','HALL','HE','PEDERSEN','POTEN','TE',
                    'O2_CM3', 'EEX', 'EEY', 'EEZ', 'ELECDEN', 'N2_CM3', 'NOP_LAM', 'NPLUS', '02P', 'O_CM3'
                    'UN', 'VN', 'WN_lev', 'HALL', 'N2D', 'N4S', 'NO', 'Ui_lev', 'Vi_lev', 'WI_LEV'],
            value='TN',
            description='TIEGCM Variable:',
            disabled=False,
            style=style1, 
            layout=layout1
            )
    def Exec_Btn_Clicked( b ):
        
        print( "Execution started..." )
        
        if TGvar.value=='All':
            thisLabel="TN"
            tgvars=['All','XNMBAR','TI','TN','OP','DEN','HALL','HE','PEDERSEN','POTEN','TE',
                    'O2_CM3', 'EEX', 'EEY', 'EEZ', 'ELECDEN', 'N2_CM3', 'NOP_LAM', 'NPLUS', '02P', 'O_CM3'
                    'UN', 'VN', 'WN_lev', 'HALL', 'N2D', 'N4S', 'NO', 'Ui_lev', 'Vi_lev', 'WI_LEV']
            RunInterpolator(os.path.splitext(tiegcm_file.value)[0],os.path.splitext(orbit_file.value)[0],tgvars,saveOption.value,thisLabel)
        else:
            thisLabel="TN"
            tgvars=[TGvar.value]
            if (TGvar.value=="XNMBAR"):
                thisLabel="Conversion Factor "
            if (TGvar.value=="TI"):
                thisLabel="Ion Temperature [K] "
            if (TGvar.value=="TN"):
                thisLabel="Neutral Temperature [K] "
            if (TGvar.value=="OP"):
                thisLabel="O+ ion [cm^-3] "
            if (TGvar.value=="DEN"):
                thisLabel="Total Density [g/cm3] "
            if (TGvar.value=="HALL"):
                thisLabel="Hall conductivity [S/m] "                
            if (TGvar.value=="HE"):
                thisLabel="Helium [mmr] " 
            if (TGvar.value=="PEDERSEN"):
                thisLabel="Pendersen condcutivity [S/m] "                 
            if (TGvar.value=="POTEN"):
                thisLabel="Electric Potential [Volts] "   
            if (TGvar.value=="TE"):
                thisLabel="Electorn Temperature [K] "     
            if (TGvar.value=="O2_CM3"):
                thisLabel="Molecular Oxygen [cm^-3] "    
            if (TGvar.value=="O2_CM3"):
                thisLabel="Molecular Oxygen [cm^-3] "                           
            if (TGvar.value=="EEX"):
                thisLabel="Electric Field East [V/cm] "
            if (TGvar.value=="EEY"):
                thisLabel="Electric Field North [V/cm] "
            if (TGvar.value=="EEZ"):
                thisLabel="Electric Field Up [V/cm] "
            if (TGvar.value=="ELECDEN"):
                thisLabel="Electron Density [cm^-3] "
            if (TGvar.value=="N2_CM3"):
                thisLabel="Molecular Nitrogen [cm^-3] "
            if (TGvar.value=="NOP_LAM"):
                thisLabel="NO+ Ion [cm^-3] "
            if (TGvar.value=="NPLUS"):
                thisLabel="N+ Ion [cm^-3] "
            if (TGvar.value=="02P"):
                thisLabel="O2+ Ion  [cm^-3] "
            if (TGvar.value=="O_CM3"):
                thisLabel="Atomic Oxygen  [cm^-3] "
            if (TGvar.value=="UN"):
                thisLabel="Neutral Zonal Wind  [cm/s] "
            if (TGvar.value=="VN"):
                thisLabel="Meridional Zonal Wind [cm/s] "
            if (TGvar.value=="WN_lev"):
                thisLabel="Vertical Zonal Wind [cm/s] "
            if (TGvar.value=="HALL"):
                thisLabel="Hall conductivity [S/m] "
            if (TGvar.value=="N2D"):
                thisLabel="N2D [mmr] "
            if (TGvar.value=="N4S"):
                thisLabel="N4S [mmr] "
            if (TGvar.value=="NO"):
                thisLabel="Nitric Oxide [mmr] "
            if (TGvar.value=="Ui_lev"):
                thisLabel="Zonal ExB velocity [cm/s] "
            if (TGvar.value=="Vi_lev"):
                thisLabel="Meridional ExB velocity [cm/s] "
            if (TGvar.value=="WI_LEV"):
                thisLabel="Vertical ExB velocity [cm/s] "
                
                
            RunInterpolator(os.path.splitext(tiegcm_file.value)[0],os.path.splitext(orbit_file.value)[0],tgvars,saveOption.value,thisLabel)
  

    def createGUI():
        MainPanel = widgets.VBox() 
        MainTab = widgets.Tab() 
        VerticalPanel = widgets.VBox()
        MapPanel = widgets.VBox()
        MainTab.children = [ VerticalPanel ]
        MainTab.set_title(0, 'Interpolator')
        MainPanel.children = [MainTab ]   
        Exec_Btn = widgets.Button (description='Run',tooltip="Click here to Interpolate",)
        Exec_Btn.style.button_color = 'Red'
        Exec_Btn.on_click( Exec_Btn_Clicked )
        VerticalPanel.children = [tiegcm_file,orbit_file,TGvar,saveOption,Exec_Btn]
    
        return MainPanel
    display( createGUI() )


def RunInterpolator(model_data_file,orbit_file,TGvar,Save=False,thisLabel="TN"):
    model=Model(model_data_file ,500,100)  #initialize model
    orbit=Orbit(orbit_file)                #initialize  orbit
    outfile=os.getenv("HOME")+"/"+os.path.basename(model_data_file)+"_Interpolated.nc"
    dtime,dlat,dlon,dalt,index,int_final=orbit.createorbit(orbit.name,model.minAltitude,model.maxAltitude,outfile,Save)
    gtime,glat,glon,glev,zg=model.readGrid(model.name)                                     #get model stats
    

    
    #select variable
    for jj in tqdm(range(len(TGvar))):
        var=model.readVar(model.name,TGvar[jj])
        print("Start TIEGCM cut...")
        sumCut = tiegcm_cut(glat,glon,glev,dtime, dlat,dlon,dalt,zg,var,thisLabel)
        print("Start Gaussian fit...")
        interpolatedData=Interpolate_Serial(glat,glon,glev,dtime, dlat,dlon,dalt,zg,var,False,thisLabel)
        interpolatedDataPlus15=Interpolate_Serial(glat,glon,glev,dtime, dlat,dlon,dalt+25,zg,var,False,thisLabel)
        
        sumGauss, min, max = GaussianFit(interpolatedData, dlat, dalt, interpolatedDataPlus15, dalt+25, 0)
        
        
        heightIntegrationPlot(sumCut, sumGauss, dlat, min, max)
        


            
            
def tiegcm_cut(glat,glon,glev,dtime, dlat,dlon,dalt,zg,var,thisLabel):
    maxlat = 0
    minlat = 0
    maxalt = 0
    minalt = 0
    
          
    for i in range(dlat.size-1):
        if (dlat[i] >= 60):
            minlat = i
            break
                
    for i in range(dlat.size-1):
        if(dlat[i]>=80):
            maxlat = i
            break
        
    for i in range(dalt.size-1):
        if (dalt[i] >=100):
            minalt = i
            break
                
    for i in range(dalt.size-1):
        if (dalt[i] <= 250.0):
            maxalt = i
            break
        
    F = np.zeros((dlat.size, dlat.size))
    dalt = np.linspace(100,250,dlat.size)
    
        
    for i in range(dlat.size-1):
        mylat = np.full(dlat.size, dlat[i])
        interpolatedData = Interpolate_Serial(glat,glon,glev,dtime, mylat,dlon,dalt,zg,var,False,thisLabel)
        #print(i)
        F[:,i] = interpolatedData*1e4
  

    
    maxvalue = F.max() 
    print(maxvalue)
    levels = np.linspace(0,maxvalue,101)
    fig= plt.figure(figsize=(15,10))
    plt.contourf(dlat[minlat:maxlat], dalt, F[:,minlat:maxlat], levels=levels, cmap="jet")
    plt.colorbar();
    plt.xlabel("Lat [deg]")
    plt.ylabel("Altitude [km]")    
    
    sumCut = np.zeros(maxlat-minlat)
    dt = 0.001

    for i in range(maxlat-minlat-5):
        sum = 0
        for j in range(dalt.size-1):
            sum = sum + F[j,minlat+i]
        sumCut[i] = sum * dt

        
    return sumCut
            
def fit(alt1, alt2, m1, m2, s, size):
    
    xm=np.array([alt1, alt2])
    fm=np.array([m1, m2])
    
    
    A = np.amax(fm)
    x0 = 120 #np.mean(xm)
    if s == 0:
        s = 0.5 * (np.amax(xm) - np.amin(xm))
    
    err = float('Inf')
    tol = 1e-6
    
    while err > tol:
        f = A * np.exp(-((xm-x0)/s)**2)
        d = fm-f
        
        #construct Z matrix
        z1 = f/A
        z2 = 2*f*(xm-x0)/s**2
        z3 = 2*f*(xm-x0)**2/s**3
        Z = np.array([z1, z2, z3])
        
        # Update coefficients
        da = np.linalg.lstsq(np.dot(Z,Z.T),np.dot(Z, d.T), rcond=None)
        
        A = A + da[0][0]
        #x0 = x0 + da[0][1]
        s = s + da[0][2]
        
        err = np.amax(np.absolute(da[0] / np.array([A, x0, s])))
        
    x = np.linspace(100, 250, size)
    f = A * np.exp(-((x-x0)/s)**2)
    
#     plt.scatter(fm, xm)
#     print(fm, xm)
#     plt.plot(f, x)
#     fig= plt.figure(figsize=(6,3))
#     plt.show()
    return f, s


    
            
            
def GaussianFit(interpolatedData, dlat, dalt, interpolatedDataPlus15, dalt2, s):
    
    maxlat = 0
    minlat = 0
    maxalt = 0
    minalt = 0
    
    for i in range(dlat.size-1):
        if (dlat[i] >= 60):
            minlat = i
            break
            
    for i in range(dlat.size-1):
        if(dlat[i]>=80):
            maxlat = i
            break
            
    for i in range(dalt.size-1):
        if (dalt[i] == dalt.min()):
            minalt = i
            break
                
    for j in range(i, dalt.size-1):
        if (dalt[j] >= 250.0):
            maxalt = j
            break            
            
    print("lats: ", dlat[minlat], dlat[maxlat])
    print("alts: ", dalt[minalt], dalt[maxalt])
    
    Y = np.linspace(100, 250, dlat.size)
    F = np.zeros((dlat.size, maxlat-minlat))
    
    for i in range(maxlat-minlat-1):
#         print(i)
        try:
            F[:,i], s = fit(dalt[minlat+i], dalt2[minlat+i], interpolatedData[minlat+i]*1e4, interpolatedDataPlus15[minlat+i]*1e4, s, dlat.size)
        except:
            break
    
    #print(max-min-1)
    
    maxvalue = F.max()
    print("gauss max" ,maxvalue)
    levels = np.linspace(0,maxvalue,101)
    
    fig= plt.figure(figsize=(15,10))
    plt.contourf(dlat[minlat:maxlat], Y, F, levels=levels, cmap="jet")
    plt.colorbar();
    plt.xlabel("Lat [deg]")
    plt.ylabel("Altitude [km]")
    plt.plot(dlat[minlat:maxlat], dalt[minlat:maxlat])
    plt.plot(dlat[minlat:maxlat], dalt2[minlat:maxlat])
    plt.show()
    
    sumGauss = np.zeros(maxlat-minlat)
    dt = 0.001
    for i in range(maxlat-minlat-5):
        sum = 0
        for j in range(Y.size-1):
            sum = sum + F[j,i]
        sumGauss[i] = sum * dt
        
    return sumGauss, minlat, maxlat

def heightIntegrationPlot(sumCut, sumGauss, dlat, min, max):
    
    fig= plt.figure(figsize=(8,8))
    plt.plot(dlat[min:max], sumGauss, 'r', label="Gaussian Fit")
    plt.plot(dlat[min:max], sumCut, 'b', label="TIEGCM cut")
    plt.legend(loc="upper right")
    plt.show()

    
            
        
  



## A sample call to the Interpolation Module:

In [7]:
LoadGUI()





Execution started...


  0%|          | 0/1 [00:00<?, ?it/s]

Start TIEGCM cut...
